In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os 
from pkg import detrend_group
os.chdir("/Users/caropark/FAO_ag_check_code")

data originally comes from df_update_dt.pkl (on local disc)... then modified to .csv.
and now fixing again

redoing the avg_2crops_n_lags(df) function in 0B.csif_proxy_fx_update

In [3]:
df = pd.read_csv("./data/yield_comparison.csv")#.drop({"yield_dt", "yield_lag", "yield_lead", "whichlag", "yield"}, axis=1)
df = df.loc[:, ~df.columns.str.contains("_log|_lead|lag", regex=True)]

,year,fao_idx,cropname,csif_og,country,gridcells,csif,name,yield_og,iso_a3
0,2000.0,11.0,Barley,0.030731,Afghanistan,43929,-0.004094,Afghanistan,596.8,AFG
1,2001.0,11.0,Barley,0.026081,Afghanistan,59230,-0.008758,Afghanistan,1000.0,AFG
2,2002.0,11.0,Barley,0.043133,Afghanistan,62662,0.008280,Afghanistan,1461.9,AFG
3,2003.0,11.0,Barley,0.039029,Afghanistan,74193,0.004163,Afghanistan,888.9,AFG
4,2004.0,11.0,Barley,0.030863,Afghanistan,74235,-0.004017,Afghanistan,920.6,AFG
...,...,...,...,...,...,...,...,...,...,...
41871,2019.0,159.0,Yams,0.283696,Venezuela (Bolivarian Republic of),139798,-0.019377,Venezuela (Bolivarian Republic of),10875.2,VEN
41872,2020.0,159.0,Yams,0.260898,Venezuela (Bolivarian Republic of),140535,-0.043140,Venezuela (Bolivarian Republic of),9657.0,VEN
41873,2021.0,159.0,Yams,0.334304,Venezuela (Bolivarian Republic of),140660,0.029300,Venezuela (Bolivarian Republic of),9955.0,VEN
41874,2022.0,159.0,Yams,0.313464,Venezuela (Bolivarian Republic of),140643,0.007493,Venezuela (Bolivarian Republic of),10114.1,VEN


In [4]:
df = detrend_group(df, "yield_og", "yield_log_dt", log_transform=True)
df = detrend_group(df, "csif_og", "csif_log_dt", log_transform= True)
df['yield_lag'] = df.groupby(['cropname', 'country'])['yield_log_dt'].shift(1)
df['yield_lead'] = df.groupby(['cropname','country'])['yield_log_dt'].shift(-1)

In [5]:
corrs = (df.groupby(["cropname", "country"])
      .apply(lambda g: g[["yield_log_dt", "yield_lag", "yield_lead"]].corrwith(g["csif_log_dt"]) if len(g) >= 2 else None).dropna())
corrs = corrs.idxmax(axis=1).to_frame().reset_index().rename({0: 'whichlag'}, axis=1)

In [6]:
df = df.merge(corrs)
df['name'] = np.where(df.whichlag == "yield_log_dt", df.country, df.country + " (" + (df.whichlag).str.replace("yield_", "") + ")")

In [9]:

df["yield_log_dt"] = np.select(
    [df["whichlag"] == "yield_lead",
    df["whichlag"] == "yield_lag"],

     [df["yield_lead"],
        df["yield_lag"]],
    default= df["yield_log_dt"])


In [12]:
df.to_csv("./data/yield_comparison.csv", index=False)